In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pyblp as blp
import torch
from torch.autograd import Variable
import torch.optim as optim
from linearmodels.iv import IV2SLS

blp.options.digits = 2
blp.options.verbose = False
nax = np.newaxis